<a href="https://colab.research.google.com/github/jachiaram/GuitarML/blob/main/GuitarML.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Guitar Machine Learning Note Classifier Model V1**

## **Install Dependencies:**
In this section tensorflow is install if it isn't already present and other dependencies used in the notebook are imported here.

In [ ]:
!pip install tensorflow==2.8.0 tensorflow-gpu==2.8.0 tensorflow-io matplotlib

In [2]:
import os
from matplotlib import pyplot as plt
import tensorflow as tf
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import LabelEncoder

## **Setting up Training and Testing Data for Model:**
In this section the csv file is read in and converted to a dataframe. Then that is split into X and y data, which is then split into training and testing data.

In [3]:
df = pd.read_csv('https://raw.githubusercontent.com/jachiaram/GuitarML/main/Guitar_features.csv')

In [4]:
y_column = 'note_names'
x_columns = ['chroma_stft', 'spectral_centroid', 'spectral_bandwidth', 'spectral_rolloff']

X = df[x_columns]
y = df[y_column]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)


label_encoder = LabelEncoder()
y_train_numeric = label_encoder.fit_transform(y_train)
y_test_numeric = label_encoder.fit_transform(y_test)

y_train_one_hot = to_categorical(y_train_numeric, num_classes=37)

In [5]:
X_train.head()

,chroma_stft,spectral_centroid,spectral_bandwidth,spectral_rolloff
1216,0.217093,682.358126,1942.144351,606.435093
44,0.226638,735.818122,1796.351101,802.737782
1906,0.200154,1445.138508,2077.487903,1967.177734
1598,0.170622,1152.060449,1635.383294,2146.492712
994,0.180103,1123.374476,2135.830164,1659.460980


In [6]:
y_train.head()

1216           25. E2
44             23. D1
1906    27. F-sharp 2
1598    29. G-sharp 2
994            33. C2
Name: note_names, dtype: object

## **Sequential Model Testing:**
In this section the model will be created, for the time being a sequential model is being used. The layers will need to be manipulated until a higher accuracy score is achieved (the goal is above 75%).


In [8]:
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Dense
from sklearn.metrics import accuracy_score

In [72]:
model = Sequential()
model.add(Dense(units=64, activation='relu', input_dim=len(X_train.columns)))
model.add(Dense(units=128, activation='relu'))
model.add(Dense(units=256, activation='relu'))
model.add(Dense(units=128, activation='relu'))
model.add(Dense(units=64, activation='relu'))
model.add(Dense(units=37, activation='softmax'))

In [73]:
from keras.optimizers import Adam
optimizer = Adam(learning_rate=0.001)
model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

In [74]:
model.fit(X_train, y_train_one_hot, epochs=500, batch_size=10)

Epoch 1/500
182/182 [==============================] - 2s 4ms/step - loss: 37.7829 - accuracy: 0.0314
Epoch 2/500
182/182 [==============================] - 1s 4ms/step - loss: 3.6357 - accuracy: 0.0275
Epoch 3/500
182/182 [==============================] - 0s 2ms/step - loss: 3.6202 - accuracy: 0.0270
Epoch 4/500
182/182 [==============================] - 0s 3ms/step - loss: 3.6468 - accuracy: 0.0237
Epoch 5/500
182/182 [==============================] - 0s 2ms/step - loss: 3.6253 - accuracy: 0.0275
Epoch 6/500
182/182 [==============================] - 0s 3ms/step - loss: 3.6446 - accuracy: 0.0308
Epoch 7/500
182/182 [==============================] - 1s 3ms/step - loss: 3.6263 - accuracy: 0.0292
Epoch 8/500
182/182 [==============================] - 1s 3ms/step - loss: 3.6186 - accuracy: 0.0292
Epoch 9/500
182/182 [==============================] - 0s 3ms/step - loss: 3.6141 - accuracy: 0.0286
Epoch 10/500
182/182 [==============================] - 1s 3ms/step - loss: 3.6107 - accur

In [1]:
from tensorflow.keras.utils import to_categorical
# Convert categorical test labels to one-hot encoding
y_test_one_hot = to_categorical(label_encoder.transform(y_test), num_classes=37)

# Evaluate the model on the test set
loss, accuracy = model.evaluate(X_test, y_test_one_hot)

print(f'Test Loss: {loss:.4f}')
print(f'Test Accuracy: {accuracy:.4f}')

NameError: name 'label_encoder' is not defined

## **Testing a Different Model Type:**
In this section I was experimenting with a Random Forest Classifier to see if I could get any better results with this model versus the other one.

In [7]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

# Assuming X_train and y_train are your training data and labels
# Assuming X_test and y_test are your test data and labels

In [8]:
# Convert string labels to numeric
label_encoder = LabelEncoder()
y_train_encoded = label_encoder.fit_transform(y_train)
y_test_encoded = label_encoder.transform(y_test)

In [17]:
# Initialize the Random Forest classifier
rf_classifier = RandomForestClassifier(max_depth=20, min_samples_split=5, min_samples_leaf=16, max_features='log2', n_estimators=300, random_state=42)

In [18]:
# Train the Random Forest classifier
rf_classifier.fit(X_train, y_train_encoded)

RandomForestClassifier(max_depth=20, max_features='log2', min_samples_leaf=16,
                       min_samples_split=5, n_estimators=300, random_state=42)

In [19]:
# Make predictions on the test set
y_pred = rf_classifier.predict(X_test)

In [20]:
# Convert predictions back to original labels
y_pred_original = label_encoder.inverse_transform(y_pred)

# Evaluate accuracy
accuracy = accuracy_score(y_test, y_pred_original)
print(f"Accuracy: {accuracy}")

Accuracy: 0.21806167400881057
